# Install required libraries

In [1]:
!pip install ultralytics opencv-python-headless torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

# Main Workflow For IOU Based Trackers

In [ ]:
import cv2
import numpy as np
import uuid
from ultralytics import YOLO

# --- Init ---

# Load YOLOv8 model trained to detect players
model = YOLO("/content/best.pt")

# Load video
cap = cv2.VideoCapture("/content/15sec_input_720p.mp4")

# Dictionary to hold active tracked players and lost ones
tracked_players = {}        # player_id: bbox
lost_players = {}           # player_id: {'last_position': (x,y), 'last_seen_frame': idx}

next_player_id = 0
max_lost_frames = 25
frame_idx = 0

# Setup output video writer
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/output_iou.mp4", fourcc, fps, (width, height))

# --- Utility Functions ---

# Get center of a bounding box
def get_center(bbox):
    x1, y1, x2, y2 = bbox
    return ((x1 + x2) // 2, (y1 + y2) // 2)

#Calculate IoU between two bounding boxes
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou



while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Object Detection using YOLO
    detections = []
    results = model(frame)[0]
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append([x1, y1, x2, y2])

    updated_players = {}
    used_ids = set()
    assigned_ids = set()

    # Match current detections with previous ones using IoU
    for det in detections:
        best_iou = 0
        best_id = None
        for pid, prev_bbox in tracked_players.items():
            if pid in used_ids:
                continue
            score = iou(det, prev_bbox)
            if score > best_iou:
                best_iou = score
                best_id = pid

        if best_iou > 0.75:
            updated_players[best_id] = det
            used_ids.add(best_id)
            assigned_ids.add(best_id)

        # Try re-identifying from lost_players based on distance
        else:
            center = get_center(det)
            reid_success = False
            for pid, info in list(lost_players.items()):
                if frame_idx - info["last_seen_frame"] > max_lost_frames:
                    continue
                lost_center = info["last_position"]
                dist = ((center[0] - lost_center[0])**2 + (center[1] - lost_center[1])**2) ** 0.5
                if dist < 40:
                    updated_players[pid] = det
                    assigned_ids.add(pid)
                    del lost_players[pid]
                    reid_success = True
                    break
            # New player ID
            if not reid_success:
                updated_players[next_player_id] = det
                assigned_ids.add(next_player_id)
                next_player_id += 1

    # Add unmatched tracked players to lost_players
    for pid in tracked_players:
        if pid not in assigned_ids:
            prev_bbox = tracked_players[pid]
            lost_players[pid] = {
                "last_position": get_center(prev_bbox),
                "last_seen_frame": frame_idx
            }

    tracked_players = updated_players

    # Draw boxes
    for pid, bbox in tracked_players.items():
        x1, y1, x2, y2 = bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f"ID: {pid}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print("Output saved to /content/output_iou.mp4")



0: 384x640 1 ball, 16 players, 2 referees, 2460.0ms
Speed: 20.1ms preprocess, 2460.0ms inference, 34.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2327.8ms
Speed: 2.9ms preprocess, 2327.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2391.2ms
Speed: 3.0ms preprocess, 2391.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3105.5ms
Speed: 3.0ms preprocess, 3105.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2269.9ms
Speed: 3.1ms preprocess, 2269.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2297.9ms
Speed: 3.0ms preprocess, 2297.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2284.6ms
Speed: 3.5ms preprocess, 2284.6ms inference, 1.1m

# Main Workflow For Color Histogram Based Trackers

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- Init ---
model = YOLO("/content/best.pt")
cap = cv2.VideoCapture("/content/15sec_input_720p.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("/content/output_histogram.mp4", fourcc, fps, (width, height))

tracked_players = {}
lost_players = {}  # {id: {'hist': histogram, 'last_seen': frame_idx}}
next_id = 0
max_lost_frames = 30
frame_idx = 0

# --- Utility Functions ---

# Compute color histogram for a detection ROI
def compute_histogram(image, bbox):
    x1, y1, x2, y2 = bbox
    roi = image[y1:y2, x1:x2]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [16, 16], [0, 180, 0, 256])
    cv2.normalize(hist, hist)
    return hist

# Compare histograms
def histogram_similarity(h1, h2):
    return cv2.compareHist(h1, h2, cv2.HISTCMP_CORREL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    detections = []
    histograms = []
    results = model(frame)[0]
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        detections.append([x1, y1, x2, y2])
        histograms.append(compute_histogram(frame, [x1, y1, x2, y2]))

    updated_players = {}
    assigned_ids = set()

    for det, hist in zip(detections, histograms):
        matched = False

        # Try matching with lost players
        for pid, info in list(lost_players.items()):
            if frame_idx - info["last_seen"] > max_lost_frames:
                continue
            sim = histogram_similarity(hist, info["hist"])
            if sim > 0.8:
                updated_players[pid] = det
                del lost_players[pid]
                assigned_ids.add(pid)
                matched = True
                break

        if not matched:
            updated_players[next_id] = det
            assigned_ids.add(next_id)
            next_id += 1

    # Track unassigned tracked_players to lost
    for pid, bbox in tracked_players.items():
        if pid not in assigned_ids:
            lost_players[pid] = {
                "hist": compute_histogram(frame, bbox),
                "last_seen": frame_idx
            }

    tracked_players = updated_players

    # Draw on frame
    for pid, bbox in tracked_players.items():
        x1, y1, x2, y2 = bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f"ID: {pid}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print("Histogram video saved to /content/output_histogram.mp4")



0: 384x640 1 ball, 16 players, 2 referees, 2331.8ms
Speed: 3.7ms preprocess, 2331.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2286.3ms
Speed: 3.1ms preprocess, 2286.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2273.6ms
Speed: 3.1ms preprocess, 2273.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2295.3ms
Speed: 3.2ms preprocess, 2295.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 3282.6ms
Speed: 5.7ms preprocess, 3282.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2289.9ms
Speed: 2.9ms preprocess, 2289.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2312.3ms
Speed: 2.9ms preprocess, 2312.3ms inference, 1.0ms 